# Data Processing

## Import modules

In [1]:
spark

In [11]:
# import SparkSession
from pyspark.sql import SparkSession

In [12]:
import pyspark.sql.functions as fn
from pyspark.sql.types import StringType,DoubleType,IntegerType

## Set spark session

In [ ]:
# only for standalone pyspark
# create spar session object
spark=SparkSession.builder.appName('data_processing').getOrCreate()

In [13]:
spark.sparkContext.appName

'PySparkShell'

## Load data

In [14]:
# Load csv Dataset 
df=spark.read.csv('data/sample_data.csv',inferSchema=True,header=True)
df.createOrReplaceTempView("dfTable")

## Inspect data

In [15]:
# columns of dataframe
df.columns

['ratings', 'age', 'experience', 'family', 'mobile']

In [16]:
# shape of dataset
df.count(),len(df.columns)

(33, 5)

In [17]:
# print dataframe schema
df.printSchema()

root
 |-- ratings: integer (nullable = true)
 |-- age: integer (nullable = true)
 |-- experience: double (nullable = true)
 |-- family: integer (nullable = true)
 |-- mobile: string (nullable = true)



In [18]:
# display fisrt few rows of dataframe
df.show()
#df.show(10)

+-------+---+----------+------+-------+
|ratings|age|experience|family| mobile|
+-------+---+----------+------+-------+
|      3| 32|       9.0|     3|   Vivo|
|      3| 27|      13.0|     3|  Apple|
|      4| 22|       2.5|     0|Samsung|
|      4| 37|      16.5|     4|  Apple|
|      5| 27|       9.0|     1|     MI|
|      4| 27|       9.0|     0|   Oppo|
|      5| 37|      23.0|     5|   Vivo|
|      5| 37|      23.0|     5|Samsung|
|      3| 22|       2.5|     0|  Apple|
|      3| 27|       6.0|     0|     MI|
|      2| 27|       6.0|     2|   Oppo|
|      5| 27|       6.0|     2|Samsung|
|      3| 37|      16.5|     5|  Apple|
|      5| 27|       6.0|     0|     MI|
|      4| 22|       6.0|     1|   Oppo|
|      4| 37|       9.0|     2|Samsung|
|      4| 27|       6.0|     1|  Apple|
|      1| 37|      23.0|     5|     MI|
|      2| 42|      23.0|     2|   Oppo|
|      4| 37|       6.0|     0|   Vivo|
+-------+---+----------+------+-------+
only showing top 20 rows



## Grouping data

In [19]:
# group by one column
df.groupBy('mobile').count().show(5)

+-------+-----+
| mobile|count|
+-------+-----+
|     MI|    8|
|   Oppo|    7|
|Samsung|    6|
|   Vivo|    5|
|  Apple|    7|
+-------+-----+



In [20]:
# sort value counts
df.groupBy('mobile').count().orderBy('count',ascending=False).show(5)

+-------+-----+
| mobile|count|
+-------+-----+
|     MI|    8|
|  Apple|    7|
|   Oppo|    7|
|Samsung|    6|
|   Vivo|    5|
+-------+-----+



In [21]:
# calculate statistical measures
df.groupBy('mobile').mean().show(5)

+-------+------------------+------------------+------------------+------------------+
| mobile|      avg(ratings)|          avg(age)|   avg(experience)|       avg(family)|
+-------+------------------+------------------+------------------+------------------+
|     MI|               3.5|            30.125|           10.1875|             1.375|
|   Oppo| 2.857142857142857|28.428571428571427|10.357142857142858|1.4285714285714286|
|Samsung| 4.166666666666667|28.666666666666668| 8.666666666666666|1.8333333333333333|
|   Vivo|               4.2|              36.0|              11.4|               1.8|
|  Apple|3.4285714285714284|30.571428571428573|              11.0|2.7142857142857144|
+-------+------------------+------------------+------------------+------------------+



In [22]:
# calculate statistical measures
df.groupBy('mobile').sum().show()

+-------+------------+--------+---------------+-----------+
| mobile|sum(ratings)|sum(age)|sum(experience)|sum(family)|
+-------+------------+--------+---------------+-----------+
|     MI|          28|     241|           81.5|         11|
|   Oppo|          20|     199|           72.5|         10|
|Samsung|          25|     172|           52.0|         11|
|   Vivo|          21|     180|           57.0|          9|
|  Apple|          24|     214|           77.0|         19|
+-------+------------+--------+---------------+-----------+



In [23]:
# calculate statistical measures
df.groupBy('mobile').max().show()

+-------+------------+--------+---------------+-----------+
| mobile|max(ratings)|max(age)|max(experience)|max(family)|
+-------+------------+--------+---------------+-----------+
|     MI|           5|      42|           23.0|          5|
|   Oppo|           4|      42|           23.0|          2|
|Samsung|           5|      37|           23.0|          5|
|   Vivo|           5|      37|           23.0|          5|
|  Apple|           4|      37|           16.5|          5|
+-------+------------+--------+---------------+-----------+



In [24]:
# calculate statistical measures
df.groupBy('mobile').min().show()

+-------+------------+--------+---------------+-----------+
| mobile|min(ratings)|min(age)|min(experience)|min(family)|
+-------+------------+--------+---------------+-----------+
|     MI|           1|      27|            2.5|          0|
|   Oppo|           2|      22|            6.0|          0|
|Samsung|           2|      22|            2.5|          0|
|   Vivo|           3|      32|            6.0|          0|
|  Apple|           3|      22|            2.5|          0|
+-------+------------+--------+---------------+-----------+



In [25]:
# use spark sql
spark.sql('''select mobile, count(*) as count from dfTable
        group by mobile''').show()

+-------+-----+
| mobile|count|
+-------+-----+
|     MI|    8|
|   Oppo|    7|
|Samsung|    6|
|   Vivo|    5|
|  Apple|    7|
+-------+-----+



In [26]:
# use spark sql
spark.sql('''select mobile, min(experience), min(age) from dfTable
        group by mobile''').show()

+-------+---------------+--------+
| mobile|min(experience)|min(age)|
+-------+---------------+--------+
|     MI|            2.5|      27|
|   Oppo|            6.0|      22|
|Samsung|            2.5|      22|
|   Vivo|            6.0|      32|
|  Apple|            2.5|      22|
+-------+---------------+--------+



In [ ]:
# Aggregation
df.groupBy('mobile').agg({'experience':'sum'}).show()

## Load and Inspect data

In [ ]:
# Load csv Dataset
rtdf=spark.read.csv('data/online_retail_dataset.csv',inferSchema=True,header=True)
rtdf.createOrReplaceTempView("rtTable")

In [ ]:
# columns of dataframe
rtdf.columns

In [ ]:
# shape of dataset
rtdf.count(),len(rtdf.columns)

In [ ]:
# print dataframe schema
rtdf.printSchema()

In [ ]:
# display fisrt few rows of dataframe
rtdf.show()
#rtdf.show(10)

In [ ]:
# counting
rtdf.select(fn.count('StockCode')).show() 

In [ ]:
# distinct count
rtdf.select(fn.countDistinct('StockCode')).show()

In [ ]:
# get minimun and maximun
rtdf.select(fn.min("Quantity"), fn.max("Quantity")).show()

In [ ]:
# Variance and Standard Deviation
rtdf.select(fn.var_pop('Quantity'), fn.var_samp('Quantity'),
        fn.stddev_pop('Quantity'), fn.stddev_samp('Quantity')).show()

In [ ]:
# use sql
spark.sql('''select count(StockCode) from rtTable''').show()

In [ ]:
# Covariance and Correlation
rtdf.select(fn.corr('InvoiceNo', 'Quantity'), fn.covar_samp('InvoiceNo', 'Quantity'),
        fn.covar_pop('InvoiceNo', 'Quantity')).show()

In [ ]:
# count with groupby
rtdf.groupBy("InvoiceNo", "CustomerId").count().show(5)

In [ ]:
# agg function
rtdf.groupBy('InvoiceNo').agg({'Quantity':'count'}).show(5)

In [ ]:
# agg function
rtdf.groupBy('InvoiceNo').agg(fn.count('Quantity').alias('quan'),
        fn.expr('count(Quantity)')).show(5)

In [ ]:
# agg function
rtdf.groupBy('InvoiceNo').agg({'Quantity':'min', 'UnitPrice':'max'}).show(5)

In [ ]:
# agg function
rtdf.groupBy('InvoiceNo').agg(fn.max('Quantity'),
        fn.min('Quantity')).show(5)

## UDF

In [28]:
# UDF
from pyspark.sql.functions import udf

### Traditional Python Function

In [ ]:
# normal function 
def price_range(brand):
    if brand in ['Samsung','Apple']:
        return 'High Price'
    elif brand =='MI':
        return 'Mid Price'
    else:
        return 'Low Price'

In [ ]:
# create udf using python function
brand_udf=udf(price_range,StringType())

In [ ]:
# apply udf on dataframe
df.withColumn('price_range',brand_udf(df['mobile'])).show(10,False)

In [ ]:
# using lambda function
age_udf = udf(lambda age: "young" if age <= 30 else "senior", StringType())

In [ ]:
# apply udf on dataframe
df.withColumn("age_group", age_udf(df.age)).show(10)

### Using Pandas UDF (Spark 2.x)

In [29]:
import pandas as pd
from pyspark.sql.functions import pandas_udf, PandasUDFType

In [ ]:
# create python function
def remaining_yrs(age):
    yrs_left=100-age
    return yrs_left

In [ ]:
# create udf using python function
length_udf = pandas_udf(remaining_yrs, IntegerType())

In [ ]:
# apply pandas udf on dataframe
df.withColumn('yrs_left', length_udf(df['age'])).show(5)

In [ ]:
# use decorator
@pandas_udf(IntegerType())
def remaining_yrs2(age):
    yrs_left=100-age
    return yrs_left

In [ ]:
# apply pandas udf on dataframe
df.withColumn('yrs_left', remaining_yrs2(df['age'])).show(5)

In [ ]:
# udf using two columns 
def prod(rating,exp):
    x=rating*exp
    return x

In [ ]:
# create udf using python function
prod_udf = pandas_udf(prod, DoubleType())

In [ ]:
# apply pandas udf on multiple columns of dataframe
df.withColumn("product", prod_udf(df['ratings'],df['experience'])).show(5)

In [30]:
# use decorator
@pandas_udf(DoubleType())
def prod2(rating,exp):
    x=rating*exp
    return x

In [31]:
# apply pandas udf on multiple columns of dataframe
df.withColumn("product", prod2(df['ratings'],df['experience'])).show(5)

+-------+---+----------+------+-------+-------+
|ratings|age|experience|family| mobile|product|
+-------+---+----------+------+-------+-------+
|      3| 32|       9.0|     3|   Vivo|   27.0|
|      3| 27|      13.0|     3|  Apple|   39.0|
|      4| 22|       2.5|     0|Samsung|   10.0|
|      4| 37|      16.5|     4|  Apple|   66.0|
|      5| 27|       9.0|     1|     MI|   45.0|
+-------+---+----------+------+-------+-------+
only showing top 5 rows



### Using Pandas UDF (Spark 3.x)

In [ ]:
# create pandas udf function
@pandas_udf('int')
def remaining_yrs3(age: pd.Series) -> pd.Series:
    yrs_left=100-age
    return yrs_left

In [ ]:
# apply pandas udf on dataframe
df.withColumn('yrs_left', remaining_yrs3(df['age'])).show(5)

In [ ]:
# create pandas udf function
@pandas_udf('double')
def prod3(rating: pd.Series, exp: pd.Series) -> pd.Series:
    x=rating*exp
    return x

In [ ]:
# apply pandas udf on multiple columns of dataframe
df.withColumn("product", prod3(df['ratings'],df['experience'])).show(5)

In [ ]:
# use spark sql 
df.selectExpr('*', 'experience*ratings as product').show(5)

## Saving file

In [ ]:
# save the dataframe as single csv 
df.coalesce(1).write.csv('data/df_data.csv', header='True', mode='overwrite')

In [ ]:
# save the data into parquet format 
rtdf.write.parquet('data/retail_dataset_parquet', mode='overwrite')

In [ ]:
# read the data from parquet format 
rtdf2=spark.read.parquet('data/retail_dataset_parquet')

In [ ]:
rtdf2.show(10)

## Outlier

In [ ]:
import numpy as np

In [ ]:
wdf = spark.read.csv('data/winequality_white.csv',sep=';',inferSchema=True,header=True)

In [ ]:
# columns of dataframe
wdf.columns

In [ ]:
# shape of dataset
wdf.count(),len(wdf.columns)

In [ ]:
# print dataframe schema
wdf.printSchema()

In [ ]:
# display fisrt few rows of dataframe
#wdf.show()
wdf.show(10)

In [ ]:
wdf.select('pH','sulphates','chlorides').summary().show()

In [ ]:
# create pandas udf function
# to detect outlier
@pandas_udf('int')
def outliers_iqr(val: pd.Series) -> pd.Series:
    quartile_1, quartile_3 = np.percentile(val, [25, 75])
    iqr = quartile_3 - quartile_1
    lower_bound = quartile_1 - (iqr * 1.5)
    upper_bound = quartile_3 + (iqr * 1.5)
    
    return pd.Series(np.where((val > upper_bound) | (val < lower_bound),1,0))

In [ ]:
wdf2 = wdf.withColumn('pH_out', outliers_iqr(wdf['pH']))

In [ ]:
wdf2.cache() # 更新
wdf2.createOrReplaceTempView("wdf2Table")

In [ ]:
wdf2.printSchema()

In [ ]:
wdf2.select('pH','pH_out').filter('pH_out==1').show(10)

In [ ]:
spark.sql('''select pH,pH_out from wdf2Table where pH_out=1 limit 10''').show()

In [ ]:
# save the data into parquet format 
wdf2.write.csv('data/wdf2', header='True', mode='overwrite')

In [ ]:
# Load csv Dataset
wdf3=spark.read.csv('data/wdf2',inferSchema=True,header=True)
wdf3.createOrReplaceTempView("wdf4Table")

In [ ]:
wdf3.filter('pH_out==1').show(15)